In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import statsmodels.api as sm
import scipy.stats as stats
from statsmodels.formula.api import ols
import sklearn as sk 
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd

df = pd.read_excel("Global_INT.xlsx", sheet_name="Global-INT")
dicti = ["continent_code",	"country_code",	"age",	"sex",	"gender_identity",	"sexual_orientation",	"height",	"weight",	"bmi",	"marital_status",	
         "student_accommodation",	"work",	"income_reportada_corrigida",	"income_categorias de renda pelo salário mínimo nacional_CLASSIFICAÇÃO",	
         "income_razão da renda pelo salário mínimo nacional",	"income_grupos de referência pelo percentil_20,40,60,80,100",	"score_food_smile",	
         "score_subs_smile",	"score_Physical Activity_smile",	"score_stress_smile",	"score_social_smile",	"score_sleep_smile",	"score_envir_smile",	
         "score_total_smile",	"sedentary_behavior",	'sedentary_2',	"gad7_score",	"gad7_severidade de sintomas"]

cat_var = ["continent_code", "country_code", "sex",	"gender_identity",	"sexual_orientation", "marital_status",	
           "student_accommodation",	"work", "income_grupos de referência pelo percentil_20,40,60,80,100", "gad7_severidade de sintomas",
           "sedentary_behavior",	'sedentary_2',]

num_var = ["age", "bmi", "score_food_smile", "score_subs_smile",	"score_Physical Activity_smile",	"score_stress_smile",	
           "score_social_smile",	"score_sleep_smile",	"score_envir_smile",	
           "score_total_smile", "gad7_score"]

df.drop(columns=["height", "weight", "income_reportada_corrigida",	
                 "income_categorias de renda pelo salário mínimo nacional_CLASSIFICAÇÃO",	
                 "income_razão da renda pelo salário mínimo nacional","income_reportada_corrigida",	
                 "income_categorias de renda pelo salário mínimo nacional_CLASSIFICAÇÃO"],
                 inplace = True)

for column in cat_var: 
    if not column == "gad7_severidade de sintomas": df[column] = df[column]-1

df.to_excel("Global_INT_15_02.xlsx", index = False)
print(df.head())

In [ ]:
import pandas as pd

df = pd.read_excel("Global_INT_15_02.xlsx")

print("Columns found:", df.columns.tolist())
print("Data preview:\n", df.head())

if "continent_code" not in df.columns:
    raise ValueError("The column 'continent_code' was not found in the file. Please check the header name.")

try:
    df["continent_code"] = df["continent_code"].astype(int)
except Exception as e:
    print("Could not convert 'continent_code' to int:", e)

continent_mapping = {
    0: "América do Sul",
    1: "América do Norte",
    2: "Oceania",
    3: "Ásia",
    4: "África",
    5: "Europa"
}

for code, continent in continent_mapping.items():
    subset = df[df["continent_code"] == code]
    output_filename = f"{continent}_MODEL.xlsx"
    subset.to_excel(output_filename, index=False)
    print(f"Created {output_filename} with {len(subset)} rows.")


In [26]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
data = pd.read_excel("Global_INT_15_02.xlsx", sheet_name='Sheet1')
df = pd.DataFrame(data)
one_hotted = ["marital_status", "gender_identity", "sexual_orientation"]

encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore") # ***ignorando dados faltantes nas one_hot***
encoded = encoder.fit_transform(df[one_hotted])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(one_hotted))
df = pd.concat([df.drop(columns = one_hotted), encoded_df], axis=1)
df.to_excel("Global_INT_One_Hot.xlsx", index=False)
#print(df.head())

#36 colunas no total pós one_hot vs 23 pré one_hot

In [ ]:
import pandas as pd
from scipy.stats import ks_2samp, chi2_contingency
from sklearn.model_selection import cross_validate, ShuffleSplit
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

#AMÉRICA DO SUL

data = pd.read_excel("América do Sul_MODEL.xlsx", sheet_name='Sheet1')

cat_col_minus_oh = ["country_code", "sex", "student_accommodation", "work", 
                    "income_grupos de referência pelo percentil_20,40,60,80,100",
                    "sedentary_behavior", 'sedentary_2']

one_hotted = ["marital_status", "gender_identity", "sexual_orientation"]                
target_col = "gad7_severidade de sintomas"

X = data.drop(columns=[target_col])
Y = data[target_col]
         
missing_data_idx = X[X.isna().any(axis=1)].index.to_numpy()
complete_data_idx = X.dropna().index.to_numpy()

shuffle_split = ShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
splits = []

for train_pos, test_pos in shuffle_split.split(complete_data_idx):

    train_idx = complete_data_idx[train_pos]  
    test_idx = complete_data_idx[test_pos]

    train_idx = pd.Index(train_idx).union(missing_data_idx)          

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    Y_train, Y_test = Y.iloc[train_idx], Y.iloc[test_idx]

    # guardando split como dictionary
    splits.append({
        "X_train": X_train,
        "X_test": X_test,
        "Y_train": Y_train,
        "Y_test": Y_test
        })
    
    onehot_transformer = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
    scaler_transformer = StandardScaler()
    
    preprocessor = ColumnTransformer(
        transformers=[
        ("onehot", onehot_transformer, one_hotted), 
        ("scaler", scaler_transformer, cat_col_minus_oh + num_var)  
                     ],
    remainder='passthrough'  # Keep the other numerical columns unchanged
    )

print(f"Splits guardados: {len(splits)}")
first_split = splits[0]
print(f"Treino: {first_split['X_train'].shape}, Teste: {first_split['X_test'].shape}")
for i, split in enumerate(splits):
    X_test = split["X_test"]  # Extract test set features
    missing_values = X_test.isna().sum().sum()  # Count total NaN values

    if missing_values > 0:
        print(f"⚠️ Split {i+1}: {missing_values} missing values found in the test set.")
    else:
        print(f"✅ Split {i+1}: No missing values in the test set.")

print(f"Treino: {first_split['X_train'].shape}, Teste: {first_split['X_test'].shape}")

knn_imputer = KNNImputer(n_neighbors=4)
for split in splits:

    X_train_inputed = pd.DataFrame(knn_imputer.fit_transform(split["X_train"]), 
                                   columns=split["X_train"].columns)
    X_test_inputed = pd.DataFrame(knn_imputer.transform(split["X_test"]), 
                              columns=split["X_test"].columns)
    
    print("Depois da inputação:")
    print(f"Valores faltantes no treino - split:", X_train_inputed.isna().sum().sum())

    for col in split["X_train"].columns:
        if col not in one_hotted + cat_col_minus_oh:
            original_values = split["X_train"][col].dropna()
            inputed_values = X_train_inputed[col]

            ks_stat, p_value = ks_2samp(original_values, inputed_values)

            if p_value < 0.05: print(f"Variável {col}  NÃO tem distribuições similares (failed null-hypothesis) - {p_value}")
            #else: print(f"Variável {col} NÃO tem distribuições similares (proved null-hypothesis) - {p_value}")

    #chi-quadrado para testar variação entre categóricas
    for col in one_hotted + cat_col_minus_oh:
        original_counts = split["X_train"][col].value_counts()
        inputed_counts = X_train_inputed[col].round().astype(int).value_counts()

        original_counts= original_counts.reindex(inputed_counts.index, fill_value=0)
        inputed_counts= inputed_counts.reindex(original_counts.index, fill_value=0)

        chi2, p_value, _, _, = chi2_contingency([original_counts, inputed_counts])

        if p_value < 0.05: print(f"Variável {col} NÃO AFETADA pela inputação (failed null-hypothesis) - {p_value}")
        #else: print(f"Variável {col} AFETADA pela inputação (proved null-hypothesis) - {p_value}")